In [ ]:
import os
import tqdm
import glob
import numpy as np
import mplhep as hep
import awkward as ak
import boost_histogram as bh
import matplotlib.pyplot as plt
from enreg.tools.models import HPS
from enreg.tools import general as g
from enreg.tools.metrics import regression_evaluator as re
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../enreg/config/", job_name="test_app"):
    cfg = compose(config_name="benchmarking")

hep.style.use(hep.styles.CMS)

In [ ]:
os.path.isdir("/home/laurits//home/laurits/HPS_recoCut0_ntuples/Z")

In [ ]:
# Merge sample_files:
SAMPLE_DIR = {
    "z": "/home/laurits/HPS_recoCut0_ntuples/Z",
    "zh": "/home/laurits/HPS_recoCut0_ntuples/ZH",
    # "qq": "/home/laurits/HPS_recoCut0_ntuples/QQ"
}

MERGED_OUTPUT_DIR = "/home/laurits/HPS_recoCut0_ntuples"
BASE_NTUPLE_DIR = "/home/laurits/ntuples/20240924_lowered_recoPtCut"


def merge_sample_files(sample_dirs, sample, output_dir):
    sample_dir = sample_dirs[sample]
    print(sample_dir)
    data = g.load_all_data(sample_dir)
    output_path = os.path.join(output_dir, f"{sample}.parquet")
    ak.to_parquet(data, output_path, row_group_size=1024)

        
def create_reco_entries(base_dir, sample, output_dir):
    sample_wcp = os.path.join(base_dir, f"{sample}*.parquet")
    data = []
    for path in tqdm.tqdm(glob.glob(sample_wcp)):
        d = ak.from_parquet(path, columns=['reco_jet_p4s', "gen_jet_tau_p4s"])
        d = ak.Array({k: d[k] for k in d.fields})
        data.append(d)
    data = ak.concatenate(data)
    data_to_save = {
        "reco_jet_pt": g.reinitialize_p4(data.reco_jet_p4s).pt,
        "gen_tau_pt": g.reinitialize_p4(data.gen_jet_tau_p4s).pt
    }
    output_dir = os.path.join(output_dir, "recoJet")
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{sample}.parquet")
    print(f"Saving to {output_path}")
    ak.to_parquet(ak.Record(data_to_save), output_path, row_group_size=1024)


for sample in SAMPLE_DIR.keys():
    merge_sample_files(SAMPLE_DIR, sample, MERGED_OUTPUT_DIR)

# for sample in SAMPLE_DIR.keys():
#     create_reco_entries(BASE_NTUPLE_DIR, sample, MERGED_OUTPUT_DIR)

In [ ]:
PT_data = g.load_all_data("/home/laurits/ml-tau-en-reg/training-outputs/20240921_recoPtCut_removed_samples/v1/jet_regression/ParticleTransformer/zh_test.parquet")
LN_data = g.load_all_data("/home/laurits/ml-tau-en-reg/training-outputs/20240921_recoPtCut_removed_samples/v1/jet_regression/LorentzNet/zh_test.parquet")
DS_data = g.load_all_data("/home/laurits/ml-tau-en-reg/training-outputs/20240921_recoPtCut_removed_samples/v1/jet_regression/DeepSet/zh_test.parquet")
hps_data = g.load_all_data("/home/laurits/HPS_recoCut0_ntuples/zh.parquet")
recoJet_data = g.load_all_data("/home/laurits/ntuples/20240924_lowered_recoPtCut/recoJet/zh.parquet")

In [ ]:
hpsPlus_mask = hps_data.true_decay_mode == hps_data.pred_decay_mode

In [ ]:
cfg.metrics.regression.ratio_plot.resolution_plot.ylim = [0, 0.2]
cfg.metrics.regression.ratio_plot.response_plot.ylim = [0.96, 1.04]

In [ ]:
evaluators = {
    "HPS":  re.RegressionEvaluator(hps_data.pred_pt, hps_data.true_pt, cfg.metrics.regression, "zh", "HPS"),
    "HPS_":  re.RegressionEvaluator(hps_data.pred_pt[hpsPlus_mask], hps_data.true_pt[hpsPlus_mask], cfg.metrics.regression, "zh", "HPS_"),
    "RecoJet":  re.RegressionEvaluator(recoJet_data.reco_jet_pt, recoJet_data.gen_tau_pt, cfg.metrics.regression, "zh", "RecoJet"),
    "PT":   re.RegressionEvaluator(PT_data.jet_regression.pred, PT_data.jet_regression.target, cfg.metrics.regression, "zh", "ParticleTransformer"),
    "LN":  re.RegressionEvaluator(LN_data.jet_regression.pred, LN_data.jet_regression.target, cfg.metrics.regression, "zh", "LorentzNet"),
    "DS":  re.RegressionEvaluator(DS_data.jet_regression.pred, DS_data.jet_regression.target, cfg.metrics.regression, "zh", "DeepSet")
}

In [ ]:
output_dir = "/home/laurits/tmp/20240923_recoPtCut_removed_plots"
os.makedirs(output_dir, exist_ok=True)
rme = re.RegressionMultiEvaluator(output_dir, cfg.metrics.regression, "zh")

In [ ]:
rme.combine_results([evaluator for evaluator in evaluators.values()])
rme.save()

In [ ]:
rme.resolution_lineplot.fig

In [ ]:
rme.response_lineplot.fig

# Decay mode performances

In [ ]:
from enreg.tools.metrics import decay_mode_evaluator as dme

In [ ]:
# data = {sample: g.load_all_data(os.path.join(cfg.base_ntuple_path, sample + ".parquet")) for sample in cfg.comparison_samples}

for algorithm, algo_info in cfg.metrics.dm_reconstruction.algorithms.items():
    for signal_sample in cfg.metrics.dm_reconstruction.signal_samples:
        sig_info_data = data[signal_sample]
        if not os.path.exists(os.path.join(algo_info.data_dir, signal_sample + ".parquet")):
            continue
        sig_data = g.load_all_data(os.path.join(algo_info.data_dir, signal_sample + ".parquet"))

        output_dir = "output_plots_dm"
        evaluator = dme.DecayModeEvaluator(sig_data.dm_multiclass.pred, sig_data.dm_multiclass.target, output_dir, signal_sample, algorithm)
        evaluator.save_performance()